# Integration Examples using QMCPy package

In this demo, we show how to use `qmcpy` for performing numerical multiple integration of two built-in integrands, namely, the Keister function and the Asian put option payoff. To start, we import the `qmcpy` module and the function `arrange()` from `numpy` for generating evenly spaced discrete vectors in the examples.

In [1]:
from qmcpy import *
from numpy import arange

## Keister Example

We recall briefly the mathematical definitions of the Keister function, the Gaussian measure, and the Sobol distribution:

* Keister integrand: $y_j = \pi^{d/2} \cos(||x_j||_2)$
    
* Gaussian true measure: $\mathcal{N}(0,\frac{1}{2})$
    
* Sobol discrete distribution: $x_j \overset{lds}{\sim} \mathcal{U}(0,1)$

The following code snippet integrates a three-dimensional Keister function numerically by creating instances of `qmcpy`'s built-in classes, `Keister`, `Gaussian`, `Sobol` and `CLTRep`, as inputs to the function `integrate()`. 

In [3]:
dim = 3
distribution = Lattice(dimension=dim, scramble=True, replications=16, seed=7, backend='MPS')
measure = Gaussian(distribution, variance=1/2)
integrand = Keister(measure)
stopper = CLTRep(distribution,abs_tol=.05)
solution,data = integrate(stopper,integrand,measure,distribution)
print(data)

Solution: 2.1659         
Keister (Integrand Object)
Lattice (Discrete Distribution Object)
	dimension       3
	scramble        1
	replications    16
	seed            7
	backend         mps
	mimics          StdUniform
Gaussian (True Measure Object)
	distrib_name    Lattice
	mean            0
	variance        0.500
CLTRep (Stopping Criterion Object)
	inflate         1.200
	alpha           0.010
	abs_tol         0.050
	rel_tol         0
	n_init          256
	n_max           1073741824
MeanVarDataRep (AccumData Object)
	replications    16
	solution        2.166
	sighat          0.011
	n_total         256
	confid_int      [ 2.164  2.168]
	time_total      0.002



## European Arithmetic-Mean Asian Put Option: Single Level 

In this example, we want to estimate the payoff of an European Asian put option that matures at time $T$. The key mathematical entities are defined as follows:

* Stock price at time $t_j := jT/d$ for $j=1,\dots,d$ is a function of its initial price $S(0)$, interest rate $r$, and volatility $\sigma$: $S(t_j) = S(0)e^{\left(r-\frac{\sigma^2}{2}\right)t_j + \sigma\mathcal{B}(t_j)}$

* Discounted put option payoff is defined as the difference of a fixed strike price $K$ and the arithmetic average of the underlying stock prices at $d$ discrete time intervals in $[0,T]$: $max \left(K-\frac{1}{d}\sum_{j=1}^{d} S(t_j), 0 \right) e^{-rT}$

* Brownian motion true measure: $\mathcal{B}(t_j) = B(t_{j-1}) + Z_j\sqrt{t_j-t_{j-1}} \;$ for $\;Z_j \sim \mathcal{N}(0,1)$

* Lattice discrete distribution: $\:\: x_j  \overset{lds}{\sim}    \mathcal{U}(0,1)$

In [ ]:
time_vector = arange(1 / 64, 65 / 64, 1 / 64)
distribution = Lattice(dimension=len(time_vector), scramble=True, replications=16, seed=7, backend='GAIL')
measure = BrownianMotion(distribution, time_vector=time_vector)
integrand = AsianCall(
    measure = measure,
    volatility = 0.5,
    start_price = 30,
    strike_price = 25,
    interest_rate = 0.01,
    mean_type = 'arithmetic')
stopper = CLTRep(distribution, abs_tol=.1)
solution,data = integrate(stopper, integrand, measure, distribution)
print(data)

## European Arithmetic-Mean Asian Put Option: Multi-Level

This example is similar to the last one except that we use Gile's multi-level method for estimation of the option price.  The main idea can be summarized as follows:

$Y_0 = 0$

$Y_1 = \mbox{ Asian option monitored at } t = [\frac{1}{4}, \frac{1}{2}, \frac{3}{4}, 1]$

$Y_2 = \mbox{ Asian option monitored at } t= [\frac{1}{16}, \frac{1}{8}, ... , 1]$

$Y_3 = \mbox{ Asian option monitored at } t= [\frac{1}{64}, \frac{1}{32}, ... , 1]$

$Z_1 = \mathbb{E}[Y_1-Y_0] + \mathbb{E}[Y_2-Y_1] + \mathbb{E}[Y_3-Y_2] = \mathbb{E}[Y_3]$

The total run time for this example is about one-third of that for the last example.

In [4]:
time_vec = [arange(1 / 4, 5 / 4, 1 / 4),
            arange(1 / 16, 17 / 16, 1 / 16),
            arange(1 / 64, 65 / 64, 1 / 64)]
dim = [len(tv) for tv in time_vec]

discrete_distrib = IIDStdGaussian(rng_seed=7)
true_measure = BrownianMotion(dim, time_vector=time_vec)
integrand = AsianCall(true_measure,
                      volatility = 0.5,
                      start_price = 30,
                      strike_price = 25,
                      interest_rate = 0.01,
                      mean_type = 'arithmetic')
stopping_criterion = CLT(discrete_distrib, true_measure, abs_tol=0.05, n_max = 1e10)
_, data = integrate(integrand, true_measure, discrete_distrib, stopping_criterion)
print(data)

Solution: 6.2548         
AsianCall (Integrand Object)
	volatility      [ 0.500  0.500  0.500]
	start_price     [30 30 30]
	strike_price    [25 25 25]
	interest_rate   [ 0.010  0.010  0.010]
	mean_type       ['arithmetic' 'arithmetic' 'arithmetic']
	exercise_time   [ 1.000  1.000  1.000]
IIDStdGaussian (Discrete Distribution Object)
	mimics          StdGaussian
BrownianMotion (True Measure Object)
	dimension       [ 4 16 64]
	time_vector     [array([ 0.250,  0.500,  0.750,  1.000])
	                array([ 0.062,  0.125,  0.188, ...,  0.875,  0.938,  1.000])
	                array([ 0.016,  0.031,  0.047, ...,  0.969,  0.984,  1.000])]
CLT (Stopping Criterion Object)
	abs_tol         0.050
	rel_tol         0
	n_max           10000000000
	inflate         1.200
	alpha           0.010
MeanVarData (AccumData Object)
	n               [ 285617.000  37842.000  5459.000]
	n_total         331990
	confid_int      [ 6.206  6.304]
	time_total      0.096

